In [14]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix

In [15]:
from CNN.layers.convolutional import Conv
from CNN.layers.pooling import MaxPooling
from CNN.layers.flatten import Flatten
from CNN.layers.dense import DenseLayer
from CNN.layers.softmax import Softmax
from CNN.layers.relu import Relu
from CNN.layers.dropout import Dropout

In [16]:
from CNN.optimizer.adam import Adam
from CNN.sequential import SequentialModel

In [ ]:
### data
x = np.load('./covid_19_data/x.npy')
y_data = np.load('./covid_19_data/y.npy')

y=np.array([[0,0,0]])
for i in y_data:
    if i == 0:
        label=np.array([1,0,0])
    elif i == 1:
        label=np.array([0,1,0]) 
    else:
        label=np.array([0,0,1])  
    label=label.reshape(1,3)
    y=np.concatenate([y,label],axis=0)
# 이게 one-hot vector 값 ( 처음에 초기화 한거 지움 )
y=np.delete(y,0,axis=0)

In [5]:
test_idx = random.sample(range(len(x)),int(0.3*len(x)))
x_test = x[test_idx,:,:]
x_train = np.delete(x, test_idx, axis=0)
y_test = y[test_idx,:]
y_train = np.delete(y, test_idx, axis=0)

In [6]:
print(y_train.shape)
print(x_train.shape)

(11973, 3)
(11973, 224, 224)


In [12]:
bs = 64  #batch size

layers = [
    Conv(num_stride=1, padding = "same", num_filter=64, filter_size=(3, 3, 1), input_shape = (bs, 224,224,1)),
    Relu(),
    Conv(num_stride=1, padding = "same", num_filter=128, filter_size=(3, 3, 64), input_shape = (bs, 224,224,64)),
    Relu(),
    MaxPooling(size=(2, 2), stride=2),
#     Dropout(dropout_rate=0.75),
    Conv(num_stride=1, padding = "same", num_filter=256, filter_size=(3, 3, 128), input_shape = (bs, 112,112,128)),
    Relu(),
    MaxPooling(size=(2, 2), stride=2),
    Conv(num_stride=1, padding = "same", num_filter=512, filter_size=(3, 3, 256), input_shape = (bs, 56,56,256)),
    Relu(),
    MaxPooling(size=(2, 2), stride=2),
    Conv(num_stride=1, padding = "same", num_filter=512, filter_size=(3, 3, 512), input_shape = (bs, 28,28,512)),
    Relu(),
    MaxPooling(size=(2, 2), stride=2),
    Conv(num_stride=1, padding = "same", num_filter=512, filter_size=(3, 3, 512), input_shape = (bs, 14,14,512)),
    Relu(),
    MaxPooling(size=(2, 2), stride=2),
    Conv(num_stride=1, padding = "same", num_filter=512, filter_size=(3, 3, 512), input_shape = (bs, 7,7,512)),
    Relu(),   
    Flatten(),
    DenseLayer.initialize(prev_num=7*7*512, after_num=4096),
    Relu(),
    Dropout(dropout_rate=0.75),
    DenseLayer.initialize(prev_num=4096, after_num=4096),
    Relu(),
    Dropout(dropout_rate=0.75),
    DenseLayer.initialize(prev_num=4096, after_num=3),
    Softmax()
]

optimizer = Adam(lr=0.003)

model = SequentialModel(
    layers=layers,
    optimizer=optimizer
)

In [13]:
epochs = 100
model.train(x_train = x_train, y_train = y_train, epochs = epochs, batch_size = bs)
model.test(x_test = x_test, y_test = y_test, batch_size = bs)


TypeError: forward() takes 2 positional arguments but 4 were given

In [ ]:
plt.plot(range(epochs), model.train_accuracy)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
plt.plot(range(epochs), model.train_loss)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()